In [567]:
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import os
import pandas as pd
import numpy as np

In [568]:
#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [569]:
#The dataframe
untouched_csv = pd.read_csv('CSV/RAW_data.csv', sep=',')

In [570]:
#replace the direction column with the corresponding degree values
direction_map = {'N': 0,'NNE': 22.5,'NE': 45,'ENE': 67.5,'E': 90,'ESE': 112.5,'SE': 135,'SSE': 157.5,
            'S': 180,'SSW': 202.5,'SW': 225,'WSW': 247.5,'W': 270,'WNW': 292.5,'NW': 315,'NNW': 337.5
        }

untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)

C:\Users\Krupam\AppData\Local\Temp\ipykernel_20392\3079875405.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)


In [571]:
#convert to Celsius
untouched_csv['Mainland Air Temp'] = round((untouched_csv['Mainland Air Temp']-32) * 5.0/9.0, 1)
untouched_csv['LBI Air Temp'] = round((untouched_csv['LBI Air Temp']-32) * 5.0/9.0, 1)


In [572]:
#rounds all colomns
untouched_csv['Humidity (%)'] = round(untouched_csv['Humidity (%)'], 1)
untouched_csv['Wind Speed (A)'] = round(untouched_csv['Wind Speed (A)'], 1)
untouched_csv['Gusting'] = round(untouched_csv['Gusting'], 1)
untouched_csv['Atmospheric Pressure (IN)'] = round(untouched_csv['Atmospheric Pressure (IN)'], 2)
untouched_csv['Precipitation Rate'] = round(untouched_csv['Precipitation Rate'], 2)
untouched_csv['Bay Temp'] = round(untouched_csv['Bay Temp'], 2)
untouched_csv['Salinity'] = round(untouched_csv['Salinity'], 2)
untouched_csv['LBI Air Temp'] = round(untouched_csv['LBI Air Temp'], 1)
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [573]:
#determines if its a onshore breeze and adds a new column for it
onshore_degrees = [180, 135, 157.5, 90, 67.5, 45, 22.5]
untouched_csv['Onshore'] = untouched_csv['Direction (A)'].isin(onshore_degrees)
untouched_csv['Onshore'] = untouched_csv['Onshore'].astype(int)



In [574]:
#thresholds
ocean_thresh = 1.0
#bay_thresh = 1.67
wind_thresh = 7.0

In [575]:
#thresholds
#ocean_thresh = 1.0
#bay_thresh = 1.67
#wind_thresh = 7.0

#untouched_csv['rolling avg'] = untouched_csv['Bay Temp'].rolling(window=24).mean()

#Boolean math to check if it is upwelling or not - OLD
#untouched_csv['bay_bool'] = (abs(untouched_csv['Bay Temp'] - untouched_csv['rolling avg']) > bay_thresh).astype(int)
#untouched_csv['ocean_bool'] = (abs(untouched_csv["Bay Temp"] - untouched_csv["Ocean Temp"]) > ocean_thresh).astype(int)
#untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
#untouched_csv["upwelling_flag"] = ((untouched_csv['bay_bool'] == 1) & (untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)
"""Explain this here"""

#Drop the temporary colomns
#untouched_csv.drop(['bay_bool', 'ocean_bool', 'big_wind', 'rolling avg'], axis=1, inplace=True)
	
	
    

'Explain this here'

In [576]:
#average of two lowest ocean points in a day
untouched_csv['ocean_min1'] = untouched_csv['Ocean Temp'].rolling(24).min()
untouched_csv['ocean_min2'] = (
    untouched_csv['Ocean Temp'].rolling(24).apply(lambda x: np.sort(x)[1], raw=False)
)
untouched_csv['ocean_min'] = (untouched_csv['ocean_min1']+untouched_csv['ocean_min2'])/2
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [577]:
#Check if its a upwelling direction
upwell_wind = [135, 165, 180, 270]
untouched_csv['upwell_wind'] = untouched_csv['Direction (A)'].isin(upwell_wind)
untouched_csv['upwell_wind'] = untouched_csv['upwell_wind'].astype(int)

In [578]:
#bools:
#   There are two tide cycles in a day, if you take the two lowest values and average them together. 
#   Taking the absolute value of the difference of the low values and the current temperature and if this is bigger than the threshold the Ocean boolean is true. 
untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
untouched_csv['upwell_wind'] = (untouched_csv['upwell_wind'] == 1).astype(int)
untouched_csv['ocean_bool'] = (abs(untouched_csv['ocean_min'] - untouched_csv['Ocean Temp']) > ocean_thresh).astype(int)
untouched_csv["upwelling_flag"] = ((untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)

In [579]:
#drop the colomns
untouched_csv.drop(['ocean_bool', 'big_wind', 'ocean_min1', 'ocean_min2', 'upwell_wind', 'ocean_min'], axis=1, inplace=True)

In [580]:
#Save this data frame to a Finished csv
#Removes first 24 hrs of data to remove NA's
untouched_csv = untouched_csv[24:]
untouched_csv.to_csv('CSV/CLEAN.csv', index=False)

#Opens a new dataframe with the Clean csv
cleancsv = pd.read_csv('CSV/CLEAN.csv')

In [581]:


#info on csv
#print(untouched_csv.info())

#Number of upwelling events
num_upwelling_events = untouched_csv['upwelling_flag'].sum()
print("Number of upwelling hours:", num_upwelling_events)


#Find which rows where upwelling
#upwelling_rows = untouched_csv[untouched_csv['upwelling_flag'] == 1]
#print(upwelling_rows)


#print(untouched_csv.to_string())
#untouched_csv.head()

Number of upwelling hours: 40


In [582]:
#Convert data into Date time and create date filter
cleancsv['Date'] = pd.to_datetime(cleancsv['Date'])
cleancsv['Date'] = cleancsv['Date'] + pd.to_timedelta(cleancsv["Hr"], unit="h")
cleancsv.drop('Hr', axis=1, inplace=True)

"""
Use this in future if data set needs specific dates
prediction = data.loc{
    (untouched_csv['Date'] > datetime(x, x, x)) &
    (untouched_csv['Date'] < datetime(x, x, x,))
}
"""


"\nUse this in future if data set needs specific dates\nprediction = data.loc{\n    (untouched_csv['Date'] > datetime(x, x, x)) &\n    (untouched_csv['Date'] < datetime(x, x, x,))\n}\n"

In [583]:
#Prepare 
data_main_air_temp = cleancsv['Mainland Air Temp']
data_humidity_per = cleancsv['Humidity (%)']
data_wind_direction = cleancsv['Direction (A)']
data_wind_speed = cleancsv['Wind Speed (A)']
data_gusting = cleancsv['Gusting']
data_pressure = cleancsv['Atmospheric Pressure (IN)']
data_rainfall = cleancsv['Precipitation Rate']
data_bay_temp = cleancsv['Bay Temp']
data_salinity = cleancsv['Salinity']
data_lbi_temp = cleancsv['LBI Air Temp']
data_ocean_temp = cleancsv['Ocean Temp']
data_onshore_flag = cleancsv['Onshore']
data_upwelling_flag = cleancsv['upwelling_flag']

#saves all data into one Numpy array
dataset = np.column_stack([
    data_main_air_temp.values,
    data_humidity_per.values,
    data_wind_direction.values,
    data_wind_speed.values,
    data_gusting.values,
    data_pressure.values,
    data_rainfall.values,
    data_bay_temp.values,
    data_salinity.values,
    data_lbi_temp.values,
    data_ocean_temp.values,
    data_onshore_flag.values,
    data_upwelling_flag.values,
])


In [584]:
#make lists for the X/Y trains to utilize later

num_cols = [
    'Mainland Air Temp', 'Humidity (%)', 'Direction (A)', 'Wind Speed (A)',
    'Gusting', 'Atmospheric Pressure (IN)', 'Precipitation Rate',
    'Bay Temp', 'Salinity', 'LBI Air Temp', 'Ocean Temp',
    'Onshore', 'upwelling_flag'
]

target_cols = ['upwelling_flag', 'Onshore', 'Wind Speed (A)', 'Direction (A)']

In [585]:
#Length of training data
datapercent = 0.95

training_data_len = int(np.ceil(len(cleancsv) * datapercent))

In [ ]:
#Scaler
scalerx = MinMaxScaler()
scalery = MinMaxScaler()
scaledx = scalerx.fit_transform(cleancsv[num_cols])
scaledy = scalery.fit_transform(cleancsv[target_cols])

training_data = scaledx[:training_data_len] #95% of all data

X_train, y_train = [], []

In [587]:
#Sliding window over last 24 hrs
for i in range(24, training_data_len):
    X_train.append(training_data[i-24:i, 0])
    y_train.append(training_data[i,0])

X_train = np.array(X_train)

y_train = np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [588]:
#Build the model
model = keras.models.Sequential()


In [589]:
#First Layer
model.add(keras.layers.LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)))

c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [590]:
#Second Layer
model.add(keras.layers.LSTM(64, return_sequences=False))

In [591]:
#3rd Layer (Dense)
model.add(keras.layers.Dense(128, activation="relu"))

In [592]:
#4th Layer (Dropout)
model.add(keras.layers.Dropout(0.5))

In [593]:
#Final Output Layer (Dense)
model.add(keras.layers.Dense(4))

In [594]:
#Put all the layers together
model.summary()
model.compile(optimizer="adam",
              loss="mae",
              metrics=[keras.metrics.RootMeanSquaredError()])

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                  │ (None, 24, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,756 (229.52 KB)

 Trainable params: 58,756 (229.52 KB)

 Non-trainable params: 0 (0.00 B)

In [595]:
#Train the model

#epochs = # of runs
#batch size = how much data is in each batch
training = model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mae/sub/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start

  File "c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\asyncio\base_events.py", line 608, in run_forever

  File "c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\asyncio\base_events.py", line 1936, in _run_once

  File "c:\Users\Krupam\miniconda3\envs\lstmenv\Lib\asyncio\ev

In [ ]:
#Prep the test data
test_data = scaledx[training_data_len-24:]
X_test, y_test = [], dataset[training_data_len:]

In [ ]:
for i in range(24, len(test_data)):
    X_test.append(test_data[i-24:i, 0])

In [ ]:
#turns test back into numpy array
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
#Make a prediction
prediction = model.predict(X_test)
prediction = scalery.inverse_transform(prediction)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


ValueError: non-broadcastable output operand with shape (136,1) doesn't match the broadcast shape (136,4)